In [13]:
import pandas as pd
import numpy as np
from Neo4jConnection import *

---

### Kết Nối Neo4j

---

Lấy đồ thị ở Neo4j

In [14]:
neo4j_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", password="123456789")
    
query = """
MATCH (n)-[r:CONNECTED]->(m)
RETURN n.name AS From, m.name AS To, r.weight AS Weight
"""
    
edges = neo4j_conn.query(query)

neo4j_conn.close()

---

### Chuyển thành dataframe

---

In [15]:
df = pd.DataFrame(edges)
df['Edge'] = df.apply(lambda row: tuple(sorted([row['From'], row['To']])), axis=1)
df_unique = df.drop_duplicates(subset=['Edge'])
df_unique = df_unique.drop(columns=['Edge'])
df_unique

,From,To,Weight
0,v1,v2,4
1,v1,v8,8
3,v2,v3,8
4,v2,v8,11
6,v3,v4,7
7,v3,v6,4
8,v3,v9,2
10,v4,v5,9
11,v4,v6,14
13,v5,v6,10


---

### Ma trận kề

---

In [16]:
vertices = sorted(set(df_unique['From']).union(df_unique['To']))
adj_matrix = pd.DataFrame(np.full((len(vertices), len(vertices)), np.inf), 
                             index=vertices, columns=vertices)

for _, row in df_unique.iterrows():
    adj_matrix.loc[row['From'], row['To']] = row['Weight']

np.fill_diagonal(adj_matrix.values, 0)  
adj_matrix

,v1,v2,v3,v4,v5,v6,v7,v8,v9
v1,0.0,4.0,inf,inf,inf,inf,inf,8.0,inf
v2,inf,0.0,8.0,inf,inf,inf,inf,11.0,inf
v3,inf,inf,0.0,7.0,inf,4.0,inf,inf,2.0
v4,inf,inf,inf,0.0,9.0,14.0,inf,inf,inf
v5,inf,inf,inf,inf,0.0,10.0,inf,inf,inf
v6,inf,inf,inf,inf,inf,0.0,2.0,inf,inf
v7,inf,inf,inf,inf,inf,inf,0.0,1.0,6.0
v8,inf,inf,inf,inf,inf,inf,inf,0.0,7.0
v9,inf,inf,inf,inf,inf,inf,inf,inf,0.0


---

### Lưu dưới dạng .csv

---

In [17]:
df_unique.to_csv('Algorithms/_Graph.csv', columns=['From', 'To', 'Weight'], index=False)